# Prototype implementation of anticipation function interface

This notebook implements a prototype interface for modelling _anticipation functions_ (AF) of non-timber values within the optimization models solved by `ws3`.

We will test usefulness of the design by implementing both bird and caribou demographic population model AFs. We will implement these AFs as linear constraints in the optimization model.

The proposed implementation automates the process of formulating the new constraints and injecting them into the optimization model. The user interface with the AF is basically to provide a string _expression_ that can be resolved by the `ws3` expression parser. This expression may refer to one or more yield curves or constants that are already built into the model. In some cases, users may need to add yield curves to an existing model for their expressions to parse correctly (e.g., we may need to add biomass volume yield curves for the bird model, which may in turn convert merchantable volume yield curves). 

The `spadesws3` module already includes a `_gen_scen_base` high-level function that automates the process of generating a (base scenario) optimization problem in a way that is compatible with the `ws3.forest.ForestModel.add_problem` method (which in turn automates the process of generating a valid Model I optimization problem and interfacing with an external solver via the lower-level optimization interface built into the `ws3.optimize` submodule). We will attempt to extend this high-level interface to include an AF component. 

Assuming that any missing yield curves have already been added to the model, adding an AF will basically involve
* defining a `cmp_c_cXXX` function functions to compile appropriate  coefficients for the constraint rows of the optimization problem matrix (given a `ForestModel` instance, a path from a DevelopmentType state tree, a string expression to evaluate, and a mask string), which ;
* adding items to `coeff_funcs` and `cgen_data` dicts inside the `_gen_scen_*` function (using matching keys). 




In [119]:
# emulate the Python environement we use in the spades_ws3 module
dat_path = '../../../input'
from spadesws3 import read_basenames, schedule_harvest_areacontrol

#basenames = read_basenames(dat_path+'/basenames.txt')
basenames = ['tsa08', 'tsa16', 'tsa24', 'tsa40', 'tsa41']
#basenames = ['tsa40']
%run -i spadesws3_params

/mnt/shared/project/ria/modules/spades_ws3/python
['/mnt/shared/project/ria/modules/spades_ws3/python', '/opt/tljh/user/lib/python36.zip', '/opt/tljh/user/lib/python3.6', '/opt/tljh/user/lib/python3.6/lib-dynload', '', '/opt/tljh/user/lib/python3.6/site-packages', '/mnt/shared/project/ws3', '/opt/tljh/user/lib/python3.6/site-packages/IPython/extensions', '/mnt/home/jupyter-gparadis/.ipython']


In [121]:
# instantiate a new ws3.forest.ForestModel instance using the bootstrap function defined in spadesws3_params.py
fm = bootstrap_forestmodel_kwargs()

copying ../../../input/tif/tsa08/inventory_init.tif ../../../input/tif/tsa08/inventory_2015.tif
copying ../../../input/tif/tsa16/inventory_init.tif ../../../input/tif/tsa16/inventory_2015.tif
copying ../../../input/tif/tsa24/inventory_init.tif ../../../input/tif/tsa24/inventory_2015.tif
copying ../../../input/tif/tsa40/inventory_init.tif ../../../input/tif/tsa40/inventory_2015.tif
copying ../../../input/tif/tsa41/inventory_init.tif ../../../input/tif/tsa41/inventory_2015.tif
bootstrap_areas tsa08 2015 6.25 7644875.0
bootstrap_areas tsa16 2015 6.25 4780950.0
bootstrap_areas tsa24 2015 6.25 6752206.25
bootstrap_areas tsa40 2015 6.25 4195350.0
bootstrap_areas tsa41 2015 6.25 2160268.75


In [100]:
# simulate_harvest(fm, basenames, 2015, mode='optimize')

## Notes on caribou demographic model and its AF

Now that we have handle to a known-working `ForestModel` instance that can solve the built-in base scenario optimization problem on one management unit (TSA 40, in north-eastern BC), we can work on integrating a caribou demographic AF.

The caribou demographic AF is a _simplified version_ of a more complex caribou demographic model, which includes some spatially-explicit components that cannot easily be expressed in the context of an aspatial area-based optimization model (such as the one we solve here using `ws3`). 

The full caribou model itself is relatively simple, conceptually: we must keep _area affected by anthropogenic disturances_ below 40% of total forest area, where an area is considered affected by anthropogenic disturbances if it is located within 500 meters of an anthropogenic disturbance that occurred less than 40 years ago (including harvest blocks, roads, mines, seismic lines, etc.). 

We cannot account for any disturbances related to the mining industry or roads in `ws3`, so we will simply ignore those in our AF and only condider area disturbed by harvesting. We also cannot easily account for the 500 meter buffer around harvested areas in the optimization model (although we _should_ be able to estimate the buffered area in a post hoc analysis of the raster output of the SDA function (e.g., using the spatial analysis functions in the `rasterio` package).

We _can_ however estimate the worst-case expansion factors based a geometric analysis, and then scale that worst-case down with a linear scaling factor that, if correctly calibrated for a given landscape, should result in a reasonably good approximation that we can use in an AF in the optimization model. We will base the prototype AF on this design.


## Implementing the caribou AF

As detailed in the previous section, we need to be able to track total _inventory_ area that is less than a given threshold age (40 years, in this example) over time (assuming this is a multi-period problem). The `ForestModel` class includes a flexible `inventory` method that is already set up to filter by age class, so this should be pretty simple.

In [101]:
horizon = 10
fm = bootstrap_forestmodel_kwargs()
fm.set_horizon(horizon)
fm.grow()

copying ../../../input/tif/tsa40/inventory_init.tif ../../../input/tif/tsa40/inventory_2015.tif
bootstrap_areas tsa40 2015 6.25 4195350.0


In [102]:
# inventory of age class 40, period 1
fm.inventory(1, age=40)

18343.75

In [71]:
# sum of inventory for age classes up to and including 40, period 1
sum(fm.inventory(1, age=age) for age in range(41))

211837.5

In [72]:
# now loop through all periods
{t:sum(fm.inventory(t, age=age) for age in range(1, 41)) for t in fm.periods}

{1: 211837.5,
 2: 193493.75,
 3: 187937.5,
 4: 184887.5,
 5: 182468.75,
 6: 178181.25,
 7: 166475.0,
 8: 157175.0,
 9: 151806.25,
 10: 149375.0}

Now we can roll something like the above code into a function that is compatible with the format required for inclusion in the `coeff_funcs` argument of the `ForestModel.add_problem` method.

In [82]:
def cmp_c_cinv(fm, path, expr, ages=None, mask=None):
    ages = fm.ages if not ages else ages 
    result = {t:sum(fm.inventory(t, age=age) for age in ages) for t in fm.periods}
    return result

In [83]:
expr = '1.'
ages = range(41)
mask = '? ? ? ?'
cmp_c_cinv(fm, expr, ages, mask)

{1: 0.0, 2: 0.0}

Now we copy the `gen_scen` functions from `spadesws3` and modify them a bit to include the caribou AF.

In [106]:
from spadesws3 import cmp_c_z, cmp_c_cflw, cmp_c_caa

def _gen_scen_boo(fm, basenames, name='boo', util=0.85, param_funcs=None, harvest_acode='harvest',  
                  tvy_name='totvol', toffset=0, obj_mode='max_hvol', target_path='./input/targets.csv',
                  max_tp=2074, cacut=None, mask=None, boo_ages=range(1, 41), boo_dthresh=0.40):
    from functools import partial
    acodes = ['null', harvest_acode]  
    vexpr = '%s * %0.2f' % (tvy_name, util)
    if obj_mode == 'max_hvol':
        sense = ws3.opt.SENSE_MAXIMIZE 
        zexpr = vexpr
    elif obj_mode == 'min_harea':
        sense = ws3.opt.SENSE_MINIMIZE 
        zexpr = '1.'
    else:
        raise ValueError('Invalid obj_mode: %s' % obj_mode)
    if not param_funcs:
        df_targets = pd.read_csv(target_path).set_index(['tsa', 'year'])
        param_funcs = {}
        param_funcs['cvcut'] = lambda bn, t: float(df_targets.loc[bn, t]['vcut']) if t <= max_tp else float(df_targets.loc[bn, max_tp]['vcut'])
        param_funcs['cabrn'] = lambda bn, t: float(df_targets.loc[bn, t]['abrn']) if t <= max_tp else float(df_targets.loc[bn, max_tp]['abrn'])
        # BOO ###
        param_funcs['caboo'] = lambda bn: fm.inventory(0, mask='%s ? ? ?' % bn) * boo_dthresh
        #########
        param_funcs['cflw_acut_e'] = lambda bn, t: df_targets.loc[bn, t]['cflw_acut_e'] if t <= max_tp else df_targets.loc[bn, max_tp]['cflw_acut_e']
        param_funcs['cgen_vcut_e'] = lambda bn, t: df_targets.loc[bn, t]['cgen_vcut_e'] if t <= max_tp else df_targets.loc[bn, max_tp]['cgen_vcut_e']
        param_funcs['cgen_acut_e'] = lambda bn, t: df_targets.loc[bn, t]['cgen_vcut_e'] if t <= max_tp else df_targets.loc[bn, max_tp]['cgen_vcut_e']
        param_funcs['cgen_abrn_e'] = lambda bn, t: df_targets.loc[bn, t]['cgen_abrn_e'] if t <= max_tp else df_targets.loc[bn, max_tp]['cgen_abrn_e']
    coeff_funcs = {'z':partial(cmp_c_z, expr=zexpr)}
    coeff_funcs.update({'cacut_%s' % bn:partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=(bn, '?', '?', '?')) for bn in basenames})
    coeff_funcs.update({'cvcut_%s' % bn:partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=(bn, '?', '?', '?')) for bn in basenames})
    # BOO ###
    coeff_funcs.update({'caboo_%s' % bn:partial(cmp_c_cinv, expr='1.', ages=boo_ages, mask=(bn, '?', '?', '?')) for bn in basenames})
    #########
    T = fm.periods
    cflw_e, cgen_data = {}, {}
    for bn in basenames:
        cgen_data.update({'cvcut_%s' % bn:{'lb':{t:param_funcs['cvcut'](bn, fm.base_year+(t-1)*fm.period_length+toffset) *
                                                   (1. - param_funcs['cgen_vcut_e'](bn, fm.base_year+(t-1)*fm.period_length+toffset)) for t in T}, 
                                         'ub':{t:param_funcs['cvcut'](bn, fm.base_year+(t-1)*fm.period_length+toffset) for t in T}}})
        # BOO ###
        cgen_data.update({'caboo_%s' % bn:{'lb':{t:0. for t in T}, 'ub':{t:param_funcs['caboo'](bn) for t in T}}})
        #########
        if cacut:
            cgen_data.update({'cacut_%s' % bn:{'lb':{t:param_funcs['cacut'](bn, fm.base_year+(t-1)*fm.period_length)*
                                                       (1. - param_funcs['cgen_acut_e'](bn, fm.base_year+(t-1)*fm.period_length)) for t in T}, 
                                             'ub':{t:param_funcs['cacut'](bn, fm.base_year+(t-1)*fm.period_length) for t in T}}})
    fm._tmp = {}
    fm._tmp['param_funcs'] = param_funcs
    fm._tmp['cgen_data'] = cgen_data
    return fm.add_problem(name, coeff_funcs, cflw_e, cgen_data=cgen_data, acodes=acodes, sense=sense, mask=mask)


def gen_scen(fm, basenames, name, util, param_funcs, toffset=0, obj_mode='max_hvol', cacut=None, mask=None, target_path='./input/targets.csv', **kwargs):
    dsp = {'boo':_gen_scen_boo}
    return dsp[name](fm, basenames, name, util, param_funcs=param_funcs, toffset=toffset, obj_mode=obj_mode, cacut=cacut, mask=mask, target_path=target_path, **kwargs)


def schedule_harvest_optimize(fm, basenames, scenario_name='base', util=0.85, param_funcs=None, 
                              target_path='./input/targets.csv', obj_mode='min_harea', mask=None, **kwargs):
    import gurobipy as grb
    p = gen_scen(fm, basenames, scenario_name, util, param_funcs=param_funcs, toffset=0, obj_mode=obj_mode, mask=mask, target_path=target_path, **kwargs)
    m = p.solve()
    if m.status != grb.GRB.OPTIMAL:
        print('Model not optimal.')
        return None
    sch = fm.compile_schedule(p)
    fm.reset_actions()
    fm.initialize_areas()
    fm.apply_schedule(sch, 
                      force_integral_area=True, 
                      override_operability=True,
                      fuzzy_age=True,
                      recourse_enabled=True,
                      verbose=False,
                      compile_c_ycomps=True)
    return sch

In [116]:
fm = bootstrap_forestmodel_kwargs()
fm.dtypes = dict(list(fm.dtypes.items())[:3])

copying ../../../input/tif/tsa40/inventory_init.tif ../../../input/tif/tsa40/inventory_2015.tif
bootstrap_areas tsa40 2015 6.25 4195350.0


In [118]:
sch = schedule_harvest_optimize(fm, basenames, scenario_name='boo', target_path=target_path, boo_dthresh=0.40)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 429 rows, 659 columns and 3771 nonzeros
Model fingerprint: 0x0445b01c
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [6e+00, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 304 rows and 302 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model
foo
ws3.opt._solve_gurobi: Model infeasible, enabling feasRelaxS mode.
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 429 rows, 1509 columns and 4621 nonzeros
Model fingerprint: 0x32a5728b
Model has 850 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 427 rows, 1507 columns, 3301 nonzeros
Presolv